In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import seaborn as sns
import pandas as pd
import os
import MDAnalysis as mda
from MDAnalysis.coordinates.GRO import GROWriter
from scipy.spatial import Voronoi

def compute_apl_voronoi(xyz_positions, plot=False):
    """
    Compute area-per-lipid (APL) using Voronoi tessellation.
    
    Parameters:
        xyz_positions (np.ndarray): shape (N, 3) array of positions
        plot (bool): whether to plot the Voronoi diagram
    Returns:
        apl_values (np.ndarray): APL for each point (in same order as input)
    """

    # Project to XY plane
    points_2d = xyz_positions[:, :2]

    # Voronoi tessellation
    vor = Voronoi(points_2d)

    # Compute areas
    apl_values = np.zeros(len(points_2d))

    for i, region_index in enumerate(vor.point_region):
        vertices = vor.regions[region_index]
        
        if -1 in vertices or len(vertices) == 0:
            apl_values[i] = np.nan  # Infinite region (at the edge), set NaN
            continue

        polygon = vor.vertices[vertices]
        x = polygon[:, 0]
        y = polygon[:, 1]
        
        # Area of polygon
        area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
        apl_values[i] = area

    if plot:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.plot(points_2d[:, 0], points_2d[:, 1], 'o')
        for region in vor.regions:
            if not region or -1 in region:
                continue
            polygon = [vor.vertices[i] for i in region]
            plt.fill(*zip(*polygon), alpha=0.4)
        plt.title('Voronoi diagram')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.show()

    return apl_values


def replicate_xy(gro_filename, n_replicas=1):
    """
    Replicate GRO positions in XY plane by periodic box shifts.
    
    Parameters:
        gro_filename (str): path to .gro file
        n_replicas (int): number of neighbors in each direction (default 1 → 1 box shift each way)
    
    Returns:
        replicated_positions (np.ndarray): array of replicated positions (N * replicas, 3)
        box (np.ndarray): box dimensions
    """

    # Load the GRO file
    u = mda.Universe(gro_filename)
    atoms = u.select_atoms('name P')
    positions = atoms.positions.copy()  # positions in Angstroms
    positions *= 0.1  # convert to nm (GROMACS uses nm)

    # Get box dimensions (in nm)
    box = u.dimensions[:3]  # [lx, ly, lz]
    box_x, box_y = box[0], box[1]

    # List to collect all images
    replicated_positions = []

    # Define shifts in XY
    shifts = []
    for i in range(-n_replicas, n_replicas+1):
        for j in range(-n_replicas, n_replicas+1):
            shifts.append((i * box_x, j * box_y))
    
    # Replicate
    for dx, dy in shifts:
        shifted = positions.copy()
        shifted[:, 0] += dx
        shifted[:, 1] += dy
        replicated_positions.append(shifted)

    # Stack all images
    replicated_positions = np.vstack(replicated_positions)

    return replicated_positions, box



def kde_contour_plot(x, y, scatter_data,
                     xlim=None, ylim=None,
                     vmin=None, vmax=None,
                     xlabel="X", ylabel="Y", title="KDE Contour Plot",
                     fill=True, cmap="viridis",
                     png_filename=None,
                     bw_adjust=1.0,
                     add_colorbar=True):
    """
    KDE-based contour plot from x, y data with control over x/y limits and colorbar scaling.
    """
    plt.figure(figsize=(6, 5))
    
    # Plot KDE and capture the returned QuadContourSet
    kde = sns.kdeplot(
        x=x,
        y=y,
        fill=fill,
        cmap=cmap,
        levels=100,   # More smoothness
        thresh=0,     # Show everything
        bw_adjust=bw_adjust
    )

    # Manually set clim (colorbar limits) if requested
    if vmin is not None or vmax is not None:
        # kde.collections is a list of PolyCollections
        for collection in kde.collections:
            collection.set_clim(vmin=vmin, vmax=vmax)

    if add_colorbar:
        mappable = kde.collections[0]
        plt.colorbar(mappable, label="Density")

    if xlim:
        plt.xlim(xlim)
    if ylim:
        plt.ylim(ylim)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.scatter(scatter_data[:,0], scatter_data[:,1], c='red', s=10, alpha=0.5)
    plt.tight_layout()

    if png_filename:
        plt.savefig(png_filename)
        plt.close()
    else:
        plt.show()


def plot_membrane_property_fromCSV(csv_filename, gro_filename, png_filename, protein):
    # Get box vectors from the last non-empty line of the .gro file
    with open(gro_filename) as fp:
        last_line = ""
        for line in fp:
            stripped = line.strip()
            if stripped:
                last_line = stripped
    #box_x, box_y = [float(val) for val in last_line.split()[:2]]
    box_x = 11.0
    box_y = 11.0
    # Read data from CSV
    x_values = []
    y_values = []
    property_values = []
    membrane_property = "Membrane Property"
    with open(csv_filename) as fp:
        for lino, line in enumerate(fp):
            if lino == 0:
                membrane_property = line.split(",")[-1].strip()
                continue
            stripped = line.strip()
            if stripped:
                parts = stripped.split(",")
                if len(parts) >= 6:
                    if parts[1] == 'upper leaflet':
                        x, y, value = float(parts[2]), float(parts[3]), float(parts[5])
                        x_values.append(x)
                        y_values.append(y)
                        property_values.append(value)
    
    kde_contour_plot(x_values, y_values, protein, xlim=(0, 11.0), ylim=(0, 11.0), vmin=0.3, vmax=1.5, fill=True, cmap="viridis_r", bw_adjust=0.5,  title="APL", png_filename=png_filename)
    #plt.scatter(protein[:,0], protein[:,1], c='blue', s=4, alpha=0.5)


def plot_membrane_property_from_csv(csv_filename, gro_filename, png_filename, protein, resolution=50j):
    # Get box vectors from the last non-empty line of the .gro file
    with open(gro_filename) as fp:
        last_line = ""
        for line in fp:
            stripped = line.strip()
            if stripped:
                last_line = stripped
    #box_x, box_y = [float(val) for val in last_line.split()[:2]]
    box_x = 11.0
    box_y = 11.0
    # Read data from CSV
    x_values = []
    y_values = []
    property_values = []
    membrane_property = "Membrane Property"
    with open(csv_filename) as fp:
        for lino, line in enumerate(fp):
            if lino == 0:
                membrane_property = line.split(",")[-1].strip()
                continue
            stripped = line.strip()
            if stripped:
                parts = stripped.split(",")
                if len(parts) >= 6:
                    if parts[1] == 'upper leaflet':
                        x, y, value = float(parts[2]), float(parts[3]), float(parts[5])
                        x_values.append(x)
                        y_values.append(y)
                        property_values.append(value)

    # Build grid for contour plot
    grid_x, grid_y = np.mgrid[0:box_x:resolution, 0:box_y:resolution]
    points = np.stack((x_values, y_values), axis=-1)
    values = np.array(property_values)
    grid = griddata(points, values, (grid_x, grid_y), method='cubic')

    # Plot
    plt.figure(figsize=(6, 5))
    contour = plt.contourf(grid_x, grid_y, grid, cmap="viridis")
    cbar = plt.colorbar(contour)
    cbar.set_label(membrane_property)
    plt.scatter(protein[:,0], protein[:,1], c='red', s=4, alpha=0.5)
    plt.title(membrane_property)
    plt.xlabel("Box X (nm)")
    plt.ylabel("Box Y (nm)")
    plt.tight_layout()
    plt.savefig(png_filename, dpi=300)
    plt.close()
    return grid

def return_grid(csv_filename, gro_filename, resolution=50j):
    # Get box vectors from the last non-empty line of the .gro file
    with open(gro_filename) as fp:
        last_line = ""
        for line in fp:
            stripped = line.strip()
            if stripped:
                last_line = stripped
    #box_x, box_y = [float(val) for val in last_line.split()[:2]]
    box_x = 11.0
    box_y = 11.0
    # Read data from CSV
    x_values = []
    y_values = []
    property_values = []
    membrane_property = "Membrane Property"
    with open(csv_filename) as fp:
        for lino, line in enumerate(fp):
            if lino == 0:
                membrane_property = line.split(",")[-1].strip()
                continue
            stripped = line.strip()
            if stripped:
                parts = stripped.split(",")
                if len(parts) >= 6:
                    if parts[1] == 'upper leaflet':
                        x, y, value = float(parts[2]), float(parts[3]), float(parts[5])
                        x_values.append(x)
                        y_values.append(y)
                        property_values.append(value)

    # Build grid for contour plot
    grid_x, grid_y = np.mgrid[0:box_x:resolution, 0:box_y:resolution]
    points = np.stack((x_values, y_values), axis=-1)
    values = np.array(property_values)
    grid = griddata(points, values, (grid_x, grid_y), method='cubic') 
    return grid

In [5]:
u = mda.Universe('/mnt/ceph/users/mmofidi/gromacs/reduced/multiple/N2/traj_continuous_v1_400_reduced.pdb', '/mnt/ceph/users/mmofidi/gromacs/reduced/multiple/N2/traj_continuous_v1_400_reduced.xtc')
GRID = []
for ts in u.trajectory[-1000::10]:
    u.atoms.write('conf.gro')
    PROT = u.select_atoms("protein and name CA").positions[:,:2]
    os.system('fatslim apl -c conf.gro -n index.ndx --export-apl-raw bilayer_prot_apl.csv')
    plot_membrane_property_fromCSV(csv_filename="bilayer_prot_apl_frame_00000.csv", gro_filename="conf.gro", png_filename="./NX/" + str(ts.frame) + ".svg", protein=PROT/10.0)
    os.system('rm -rf *.old')


/mnt/home/asahoo/venv/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:203: UserWarning: Cannot write lock/offset file in same location as {self.filename}. Using slow offset calculation.
  warnings.warn(f"Cannot write lock/offset file in same location as "
/mnt/home/asahoo/venv/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:259: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/mnt/ceph/users/mmofidi/gromacs/reduced/multiple/N2/.traj_continuous_v1_400_reduced.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

Running command: 'apl'... This may take some time, be patient!
Analysis will be performed using 32 threads.
Analysing frame     1/    1 (time: 0 ps)... done in 18 ms (Remaining: 0 s)
Results:
Average values over 1 processed frames:
Area per lipid: Membrane=0.694±0.000nm^2 - Lower leaflet=0.666±0.000nm^2 - Upper leaflet=0.724±0.000nm^2
Area: Membrane=117.932±0.000nm^2 - Lower leaflet=117.161±0.000nm^2 - Upper leaflet=118.702±0.000nm^2

DEBUG!
'apl' command executed in 89.398 ms (CPU)
Goodbye!
FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

Running command: 'apl'... This may take some time, be patient!
Analysis will be performed using 32 threads.
Analysing frame     1/    1 (time: 0 ps)... done in 17 ms (Remaining: 0 s)
Results:
Average values over 1 processed frames:
Area per lipid: Membran

In [8]:
u = mda.Universe('/mnt/ceph/users/mmofidi/gromacs/reduced/bound/N1/traj_continuous_v1_400_reduced.pdb', '/mnt/ceph/users/mmofidi/gromacs/reduced/bound/N1/traj_continuous_v1_400_reduced.xtc')
GRID = []
for ts in u.trajectory[-1000::10]:
    u.atoms.write('conf.gro')
    PROT = u.select_atoms("protein and name CA").positions[:,:2]
    os.system('fatslim apl -c conf.gro -n index.ndx --export-apl-raw bilayer_prot_apl.csv')
    plot_membrane_property_fromCSV(csv_filename="bilayer_prot_apl_frame_00000.csv", gro_filename="conf.gro", png_filename="./N1/" + str(ts.frame) + ".svg", protein=PROT/10.0)
    #grid = return_grid(csv_filename="bilayer_prot_apl_frame_00000.csv", gro_filename="conf.gro", resolution=50j)
    #GRID.append(grid)
    os.system('rm -rf *.old')


/mnt/home/asahoo/venv/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:203: UserWarning: Cannot write lock/offset file in same location as {self.filename}. Using slow offset calculation.
  warnings.warn(f"Cannot write lock/offset file in same location as "
/mnt/home/asahoo/venv/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:259: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/mnt/ceph/users/mmofidi/gromacs/reduced/bound/N1/.traj_continuous_v1_400_reduced.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

Running command: 'apl'... This may take some time, be patient!
Analysis will be performed using 32 threads.
Analysing frame     1/    1 (time: 0 ps)... done in 37 ms (Remaining: 0 s)
Results:
Average values over 1 processed frames:
Area per lipid: Membrane=0.687±0.000nm^2 - Lower leaflet=0.665±0.000nm^2 - Upper leaflet=0.710±0.000nm^2
Area: Membrane=116.711±0.000nm^2 - Lower leaflet=116.980±0.000nm^2 - Upper leaflet=116.442±0.000nm^2

DEBUG!
'apl' command executed in 109.284 ms (CPU)
Goodbye!
FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

Running command: 'apl'... This may take some time, be patient!
Analysis will be performed using 32 threads.
Analysing frame     1/    1 (time: 0 ps)... done in 220 ms (Remaining: 0 s)
Results:
Average values over 1 processed frames:
Area per lipid: Membr

In [3]:
u = mda.Universe('/mnt/ceph/users/mmofidi/gromacs/reduced/bound/N2/traj_continuous_v1_400_reduced.pdb', '/mnt/ceph/users/mmofidi/gromacs/reduced/bound/N2/traj_continuous_v1_400_reduced.xtc')
GRID2 = []
for ts in u.trajectory[-1000::10]:
    u.atoms.write('conf.gro')
    PROT = u.select_atoms("protein and name CA").positions[:,:2]
    os.system('fatslim apl -c conf.gro -n index.ndx --export-apl-raw bilayer_prot_apl.csv')
    plot_membrane_property_fromCSV(csv_filename="bilayer_prot_apl_frame_00000.csv", gro_filename="conf.gro", png_filename="./N2/" + str(ts.frame) + ".png", protein=PROT/10.0)
    grid = return_grid(csv_filename="bilayer_prot_apl_frame_00000.csv", gro_filename="conf.gro", resolution=50j)
    GRID2.append(grid)
    os.system('rm -rf *.old')

/mnt/home/asahoo/venv/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:203: UserWarning: Cannot write lock/offset file in same location as {self.filename}. Using slow offset calculation.
  warnings.warn(f"Cannot write lock/offset file in same location as "
/mnt/home/asahoo/venv/lib/python3.8/site-packages/MDAnalysis/coordinates/XDR.py:259: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/mnt/ceph/users/mmofidi/gromacs/reduced/bound/N2/.traj_continuous_v1_400_reduced.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

Running command: 'apl'... This may take some time, be patient!
Analysis will be performed using 32 threads.
Analysing frame     1/    1 (time: 0 ps)... done in 24 ms (Remaining: 0 s)
Results:
Average values over 1 processed frames:
Area per lipid: Membrane=0.675±0.000nm^2 - Lower leaflet=0.652±0.000nm^2 - Upper leaflet=0.700±0.000nm^2
Area: Membrane=114.756±0.000nm^2 - Lower leaflet=114.773±0.000nm^2 - Upper leaflet=114.739±0.000nm^2

DEBUG!
'apl' command executed in 84.011 ms (CPU)
Goodbye!
FATSLiM - Fast Analysis Toolbox for Simulations of Lipid Membranes
version 0.2.2
Copyright (c) 2013-2016 Sébastien Buchoux

Running command: 'apl'... This may take some time, be patient!
Analysis will be performed using 32 threads.
Analysing frame     1/    1 (time: 0 ps)... done in 28 ms (Remaining: 0 s)
Results:
Average values over 1 processed frames:
Area per lipid: Membran

In [3]:
GRID = np.array(GRID)

In [38]:
replicated_positions, box = replicate_xy('conf.gro', n_replicas=1)

In [39]:
compute_apl_voronoi(replicated_positions, plot=False)

array([0.29821737, 0.40246787, 0.33013476, ..., 0.31855077, 0.40329385,
       0.27634355])

In [42]:
len(replicated_positions)/9

340.0

In [41]:
box

array([107.6791, 107.6791, 136.8677], dtype=float32)

In [ ]:
-1 -1
-1 0
-1 1
0 -1
0 0